# Solving the nearest point on curve problem

A Jupyter Notebook description of the article published in Graphic Gems by Andrew S. Glassner. Contributions from Philip J. Schneider, the University of Geneva.

by Darren R. Starr

## Introduction

This is an attempt to describe mathematically and in code the algorithm presented by Philip J. Schneider in the book [Graphic Gems](http://www.realtimerendering.com/resources/GraphicsGems/) in 1990.

I've been using this algorithm for years without really understanding how it worked. I've become quite obsessed with Bezier curves because of it. Now, after watching nearly 2,000 videos on [Khan Academy](http://khanacademy.org) as well as drinking more than a few coffees with physicists, I find the algorithm to be quite simple to understand. Or course, when I was just a lowly programmer rather than a mathematics geek, it was really confusing. I'm hoping I can dumb the algorithm down to a level even I could have understood a few years ago.

While writing, I will attempt to explain the math. Most of it is actually basic algebra as any kid could have/should have learned by the 10th grade. Some of it requires understanding a little trigonometry. And the core ability to understand it requires knowing what a derivative is and there's even a dirty trick or two thrown in from linear algebra.

If you don't know what a derivative is... or you just really like watching incredibly brilliant women teach advanced math in a way that even a moron should understand, check out [NancyPi at Youtube](https://www.youtube.com/watch?v=-ktrtzYVk_I), in 15 minutes you'll be like "wow... I didn't think it was that simple".

As for the linear algebra. If you ever learned in trig that

TODO: make a better image with proper case on labels
![law of cosines](lawofcosines.png)

$$ c^2 = a^2 + b^2 - 2ac\cos{\theta}$$

When the same calculation is shown in linear algebra it says

$$\color{red}{\lvert\overrightarrow{c}\rvert^2 =\lvert\overrightarrow{a}-\overrightarrow{b}\rvert^2 =}  \lvert\overrightarrow{a}\rvert^2 + \lvert\overrightarrow{b}\rvert^2 - 2\lvert\overrightarrow{a}\rvert\lvert\overrightarrow{b}\rvert\cos{\theta}$$

which reduces down to 

$$\lvert\overrightarrow{a}\rvert\lvert\overrightarrow{b}\rvert\cos{\theta}
= \overrightarrow{a}\cdot{\overrightarrow{b}}$$

That's the "dot product" which in terms I would have understood before 4 months and a thousand math videos would look like

$$\overrightarrow{a}\cdot{\overrightarrow{b}} = a_x b_x + a_y b_y$$

I've actually written a [full proof of this which is available on my Github](https://github.com/darrenstarr/JupyterNotebooks/blob/master/Dot%20Product%20(law%20of%20cosines)%20Proof.ipynb)

## Description of the algorithm

The algorithm itself depends on understanding that a line drawn from a given point, directly to the nearest point on a curve will be precisely 90 degrees (the normal) relative to the tangent of the curve at the point where it meets.

TODO: Insert graphics here showing this fact

It's also important to realize that it's not actually guaranteed that there is only a single point which is closest.

Consider if you were to draw a quarter of a circle which is easily represented by a Bezier curve. Now find the shortest distance from the center of the circle to the curve.

TODO: Insert graphic here showing this fact

The number of possible "shortest points in this case is actually infinite. Therefore, any point is as good as another... or maybe all points are wrong as there is no shortest point. This depends on your specific need to find it.

For that fact, maybe the result of the shortest distance to the curve should be represented as a list of ranges along the curve.

For the purpose of the book as well as this article, we will represent the solution as follows:

$$[Q(t) - P] \cdot{Q'(t)} = 0$$

What this says is that the "dot product" [Khan Academy](https://www.khanacademy.org/math/linear-algebra/vectors-and-spaces/dot-cross-products/v/vector-dot-product-and-vector-length) of a function minus the point is equal to zero.

It is more correct in my opinion to say

$$[Q(t) - P] \cdot{Q'(t)} = \cos{90^\circ}$$

Before we go further, it's important to see what function $Q(t)$ is equal to. For now, we'll represent it as what's called "The Bernstein-Bezier polynomial". Which is pretty much just the normal way which we write the Bezier curve. When we see it in code, we typically see what's called the "explicit form".

$$Q(t) = \sum_{i=0}^{n}V_iB_i^n(t), t\in{[0,1]}$$

### From math nerd to code nerd.

This is REALLY mathy, but it basically says

$\sum_{i=0}^{n}$ which is

```python
for i in range(0,n):
```

And $n$ is the number of degrees of the function. So a quadratic (ex $x^2+x-1=0$) would have 2 degrees and a cubic (ex $x^3-x^2+x-1=0$ would have 3 degrees.

Since I'm really only interested specifically in "Cubic Beziers", I'll focus entirely on strictly 3 degrees here. That will require a total of 4 points to draw.

$V$ is an array of (in our case 4) control points for example

```python
V = [
    [0,0],
    [0.1,0.66],
    [0.91,0.51],
    [1,1]
]
```

If we were to expand the equation out to start looking a little easier to read, we would get

$$Q(t) = V_0B_{i=0}^{n=3}(t) + V_1B_{i=1}^{n=3}(t) + V_2B_{i=2}^{n=3}(t) + V_3B_{i=3}^{n=3}(t)$$

This was created by goofing at [Lea Verou's really nifty curve generator](https://cubic-bezier.com/#.1,.66,.91,.51) which is a great toy for the web browser.

Then there's the function which is used a few times here called $B_i^n$ which in programmer terms would look something like B(i,n) that is then multiplied by t.

Don't worry, we'll put it altogether as well go along and even write the code

So, now we need to define what the function `B` is

$$B_i^n(t) = {n \choose i}t^i(1-t)^{n-i}, i=0,...,n,$$

So this is a little bit hairy. The good news is we're working only on values of $i$ from 0-3.

### Picky Choosy

The funky looking ${n \choose i}$ is pronounces "$n$ choose $i$" and translates into 

$$\frac{n!}{(n-i)!i!}$$

So, $n$ remains $3$ in our case and $i$ counts from $0$ to $3$ which gives us

$$\frac{3!}{(3-i)!i!}$$

or

$$\frac{n!}{(n-i)!i!}=\frac{3!}{(3-0)!0!} = \frac{3\times{2}\times{1}}{(3\times{2}\times{1})\times{1}} = \frac{6}{(6)\times{1}} = 1,\quad n=3,i=0$$

$$\frac{n!}{(n-i)!i!}=\frac{3!}{(3-1)!1!}=\frac{3!}{(2)!1!} = \frac{3\times{2}\times{1}}{(2\times{1})\times{1}} = \frac{6}{(2)\times{1}} = 3,\quad n=3,i=1$$

$$\frac{n!}{(n-i)!i!}=\frac{3!}{(3-2)!2!}=\frac{3!}{(1)!2!} = \frac{3\times{2}\times{1}}{(1)(2\times{1})} = \frac{6}{(1)\times{2}} = 3,\quad n=3,i=2$$

$$\frac{n!}{(n-i)!i!}=\frac{3!}{(3-3)!3!}=\frac{3!}{(0)!3!} = \frac{3\times{2}\times{1}}{(1)(3\times{2}\times{1})} = \frac{6}{(1)\times{6}} = 1,\quad n=3,i=3$$

So, we learn that when $n=3$, then the 4 possible values for ${n \choose i}$ are 1, 3, 3, and 1.

So expanding out all 4 possible values of $B_i^n(t)$ we find the following

$$B_0^3(t) = (1)t^0(1-t)^{3-0} = t^0(1-t)^{3} = (1-t)^3$$

$$B_1^3(t) = (3)t^1(1-t)^{3-1} = 3t(1-t)^2$$

$$B_2^3(t) = (3)t^2(1-t)^{3-2} = 3t^2(1-t)^{1} = 3t^2(1-t)$$

$$B_3^3(t) = (1)t^3(1-t)^{3-3} = t^3(1-t)^{0} = t^3$$

### Plugging it together

So, if we revisit the Berenstein Bezier polynomial which was

$$Q(t) = \sum_{i=0}^{n}V_iB_i^n(t),\quad t\in{[0,1]}$$

Which we expanded to

$$Q(t) = V_0B_{i=0}^{n=3}(t) + V_1B_{i=1}^{n=3}(t) + V_2B_{i=2}^{n=3}(t) + V_3B_{i=3}^{n=3}(t),\quad t\in{[0,1]}$$

We can plug in our values for $B_i^n$ and get the following

$$Q(t) = V_0(1-t)^3 + V_{1}3t(1-t)^2 + V_{2}3t^2(1-t) + V_{3}t^3,\quad t\in{[0,1]}$$

And to just move the numbers around a little so it looks more like it does on [Wikipedia](https://en.wikipedia.org/wiki/B%C3%A9zier_curve#Cubic_B%C3%A9zier_curves) we'll just change the order of the terms of multiplication

$$Q(t) = (1-t)^3V_0 + 3t(1-t)^2V_{1} + 3t^2(1-t)V_{2} + t^3V_{3},\quad t\in{[0,1]}$$

Now, to make it clear, $V_0$, $V_1$, $V_2$, and $V_3$ are all "vectors" which a programmer nerd might know as a matrix... or maybe a point.. or a coordinate to make things really easy. And we know that our points are defined above as an array. But let's expand out the equation a little more to make it look a little more programmery.

$$Q(t)
= (1-t)^3\begin{bmatrix}x_0\\y_0\end{bmatrix}
+ 3t(1-t)^2\begin{bmatrix}x_1\\y_1\end{bmatrix}
+ 3t^2(1-t)\begin{bmatrix}x_1\\y_1\end{bmatrix}
+ t^3\begin{bmatrix}x_2\\y_2\end{bmatrix}
,\quad t\in{[0,1]}
$$

Which is still a bunch of illegible mathematical garbage. So, let's go a little further and break this into one function for $x$ and one for $y$.

$${
\color{red}{x = (1-t)^3x_0 + 3t(1-t)^2x_{1} + 3t^2(1-t)x_{2} + t^3x_{3}}\\
\color{blue}{y = (1-t)^3y_0 + 3t(1-t)^2y_{1} + 3t^2(1-t)y_{2} + t^3y_{3}}
}\Bigg\vert t\in{[0,1]}
$$


### What time is it?

We've seen this $t$ variable all this time and it's made no sense unless you know what $t\in{[0,1]}$ means. And I know many people do, but this is the real heart of bezier curves. $t$ is a variable that stretch from $0$ through $1$ for all the infinite possible values of $0$ to $1$ including $0$ and $1$. I guess it would be like

```python
for t in range(0,1)
```

Except, instead of counting in ones, it is literally every possible value. If you were to picture an old fashioned analog volume dial, $0$ would be completely to the left and $1$ would be completely to the right. Getting from $0$ to $1$ requires turning the dial through an infinite number of values to get from $0$ to $1$. It's not like my neighbor's accelerator peddle in their car which only has completely on and completely off.

I guess it's more like

```python
for t in range(0.0, 1.0, 1.0/math.inf)
```

*I'm giggling as I write this since I imagine it would take a REALLY long time for that loop to complete... at least when running on anything other than a quantum computer*

This value of $t$ is time. Even though this math is generally geometry from our perspective, it's expressed in terms of calculus which is pretty much the measurement of change over time. Therefore, $t$ is the magical variable dedicated to expressing time. Just like $x$ goes from left to right, $y$ goes from bottom to top, $z$ goes from front to back, $t$ goes from earlier to later.


### So what does this look like

I've added a Geogebra implementation of the formula above which is interactive and you can play with it. [Here's the link to it.](https://www.geogebra.org/calculator/seexfgm8)

![Curve in terms of x and y](PlotOfCurveInTermsOfXandY.png)

As you can see from the plot above, a bezier curve in two dimension is really nothing more than combining a the plot of a cubic polynomial on the $x$ axis with the plot of a cubic polynomial on the $y$ axis.

What makes this different than normal 9th grade math is that instead of plotting a function of $x$ like the classical $f(x) = ax^3+bx^2+cx+d$, we see that the values of $a$, $b$, $c$, and $d$ are all either the $x$ or $y$ coordinates of points $V_0$, $V_1$, $V_2$, and $V_3$ and the equation instead is

$$f(t) = at^3 + bt^2 + ct + d,\quad 0<=t<=1$$

There is of course more to it than that... but let's not get too carried away :)

## If you squint your eyes and look at it on a slant

So, in standard geometry, we're pretty used to the concept of a slope. It's nothing more than a relationship between the rise of a function to the run of the function. Before the nifty term "rise over run" became common place in schools, we often just memorized the formulas which should look pretty familiar

We know that a line is represented mathematically as

$$y=mx+b$$

Where $x$ and $y$ are coordinates and $b$ is the offset from the $x$ axis along the $y$ axis. And $m$ is the slope of the line.

The slope formula describes how much does the line increase or decrease in the $y$ direction for each positive step in the $x$ direction. So, we write it as

$$m = \frac{\Delta{y}}{\Delta{x}} = \frac{y_2-y_1}{x_2-x_1}$$

Where $(x_1, y_1)$ and $(x_2, y_2)$ are two points along the line.

Well that was a fun refresher.

### The first derivative of a function

When you're trying to find the slope of any point of a function, we use the first derivative of the function.

So let's for the moment assume we use a really simple quadratic function like the following :

$$f(x)=2x^2+3x-5$$

The first derivative of that function would be:

$$f'(x)=4x+3$$

So, let's say we wanted to find out exactly what the slope fo the function $f(x)$ is when $x=1$

$$f'(1)=4(1)+3 = 5+3 = 8$$

So, we learn that the slope of the function $f(x)$ when $x=1$ is a positive slope of 8. You can see this below or look closer at it on [Geogebra](https://www.geogebra.org/calculator/vx5ykmjc)

![First Derivative Example](FirstDerivativeExample.png)

And if you want to know how to find the first derivative of a function, this is generally a huge topic. But for the simple stuff, you can see the [Power Rule at Khan Academy](https://www.khanacademy.org/math/ap-calculus-ab/ab-differentiation-1-new/ab-2-5/v/power-rule)

### The first derivative of the cubic bezier function

Following the forms used in Graphics Gems, the Bernstein-Bezier form of the derivative would be:

$$Q'(t)=n\sum_{i=0}^{n-1}(V_{i+1}-V_i)B_i^{n-i}(t)$$

at a later point, I may come back and spell out the steps, but for now, I'll just show the formula above written in "explicit form"

$$Q'(t)=3(1-t)^2(V_1-V_0)+6(1-t)t(V_2-V_1)+3t^2(V_3-V_2)$$

It's quite a bit smaller, and when it's calculated out, there is a slope for both the $x$ as well as the $y$. This should be more than a little confusing for everyone. In fact, this and one more thing we'll encounter later are why I started writing this paper. I wanted an excuse to visualize it.

![Seeing the slope](SeeingTheSlope.png)

This was a particularly fun plot to make. You can play with it at [Geogebra](https://www.geogebra.org/calculator/rz9rv8gq) where it's interactive and you can change the value of $t$ and watch it in action.

## The math starts getting pretty heavy

So, now that we've gotten to the point where we can underrstand both the initial formula for the Bezier curve as well as the slope generated by the first derivative, we can get back into the real stuff. Remember, the algorithm we're trying to represent is:

$$[Q(t)-P)\cdot{Q'(t)]=0$$

is the goal.

In order to solve this problem, we need to do some unintuitive expansion. I'll document it so it makes sense in hindsight, but honestly, I always consider it to be just short of magic when someone adds an extra step or substitution that I would have never seen myself. Though I'm sure that in this case, it would become clear if you consider they solved the equation and then tried to simplify their documentation after the fact by employing these methods.

So, let's make two new functions which are really just a way of writing what was above a little differently.

$$Q_1(t) = Q(t) - P$$

$$Q_2(t) = Q'(t)$$

This allows us to right the equation as 

$$Q_1(t)\cdot{Q_2(t)} = 0$$

### Expanding the first part

So, we want to expand the equation so that we can eventually solve it. Keep in mind that we don't have to bring everything into explicit form like we did earlier, though there are times where this could make life easier to understand. The Graphic Gems book however has eliminated some of those steps when they weren't needed.

$$Q_1(t) = Q(t) - P$$

$$=\sum_{i=0}^{n}V_iB_i^n(t) - P$$

$$=\sum_{i=0}^{n}c_iB_i^n(t)$$

where $c_i=V_i-P$

WHAT!!!

Really, this was where the book lost me and left me drooling a bit down my chin. In fact, this is precisely the kind of thing which never ever ever made sense to me. And I feel it's necessary to graph it and see what happens.

![Translate Curve by -P](TranslateQbyC.png)

So what we did was make a new set of 4 points called $c_0$, $c_1$, $c_2$, and $c_3$ which eliminated the point $P$ from the equation. If you visit the [Geogebra Page](https://www.geogebra.org/calculator/mxhszwrc) you can play with the points and the slider to see that the curve is still the same size, shape, etc. Also, the values of $t$ remain the same. Therefore the slope of $Q(t)$ at any given value of $t$ will be the same even after the translation.

You can also tell by looking at the faint blue lines that the relationship between $P$ and point $Q(t)$ is exactly the same as the relationship between the origin and the corresponding point on the translated curve.

Somehow when I was studying algebra, I DID NOT expect that distributing the $-P$ against the points of $V$ would be a perfect translation.

TODO : Explain in more detail why it makes sense once you digest Darren.

### Substituting the second part

We started with:

$$Q_2(t)=n\sum_{i=0}^{n-1}(V_{i+1}-V_i)B_i^{n-1}$$

and we saw the explicit form of this function is

$$Q'(t)=3(1-t)^2(V_1-V_0)+6(1-t)t(V_2-V_1)+3t^2(V_3-V_2)$$

Since this is a cubic polynomial, we know that $n=3$ so if we rewrite it a little by factoring $n$ back out we can see what why what we're going to do next makes sense.

$$Q'(t)=3[(1-t)^2(V_1-V_0)+2(1-t)t(V_2-V_1)+t^2(V_3-V_2)]$$

We have 3 steps here. and we should be able to see that for the three "loops" as we'd call it in programming, that

$$V_{i+1}-V_i$$

Comes out to

$$V_1-V_0$$

$$V_2-V_1$$

$$V_3-V_2$$

Which should look like a pretty familiar pattern to most programmers

```python
V[1]-V[0]
V[2]-V[1]
V[3]-V[2]
```

All we're doing is finding the difference between each value in an array.

Also, we should see that

$$B_i^{n-i}$$

is pretty much calling function $B$ with parameters $n$ and $i$

```python
B(n, i)
```

So as programmers if we expand it out, it would look something like

```python
n * ((V[1]-V[0]) * B(2, 0) + (V[2]-V[1]) * B(2, 1) + (V[3]-V[2]) * B(2, 2))
```

So... what if we put the value of $n$ back in?

$$Q'(t)=3((1-t)^2(V_1-V_0))+3(2(1-t)t(V_2-V_1))+3(t^2(V_3-V_2))$$

and we move things around just a little bit

$$Q'(t)=3(V_1-V_0)(1-t)^2+3(V_2-V_1)2(1-t)t+3(V_3-V_2)t^2$$

it would look a lot like the code

```python
n * (V[1]-V[0]) * B(2, 0) + n * (V[2]-V[1]) * B(2, 1) + n * (V[3]-V[2]) * B(2, 2)
```

So we can easily believe that the math would look like

$$Q_2(t)=n\sum_{i=0}^{n-1}(V_{i+1}-V_i)B_i^{n-1}$$

$$=\sum_{i=0}^{n-1}n(V_{i+1}-V_i)B_i^{n-1}$$

And the book suggests we move the whole chunk of $n(V_{i+1}-V_i)$ out of the function for now to make the steps we're doing now a little easier. So:

$$Q_2(t)=n\sum_{i=0}^{n-1}(V_{i+1}-V_i)B_i^{n-1}$$

$$=\sum_{i=0}^{n-1}n(V_{i+1}-V_i)B_i^{n-1}$$

$$=\sum_{i=0}^{n-1}d_iB_i^{n-1}$$

where $d_i=n(V_{i+1}-V_i)$

Wow! This is like refactoring code into their own functions. For now, let's run with it.

### Writing the abbreviated form and making it a bigger mess before the end

Now that we have the following

$$Q_1(t)=\sum_{i=0}^{n}c_iB_i^n(t)$$

$$Q_2(t)=\sum_{i=0}^{n-1}d_iB_i^{n-1}$$

where

$$c_i=V_i-P$$

$$d_i=n(V_{i+1}-V_i)$$

we can rewrite the original equation as

$$0=\sum_{i=0}^{n}c_iB_i^n(t) \cdot{\sum_{j=0}^{n-1}d_jB_j^{n-1}(t)}$$

And I've been holding off for some time on this... most of us who never really went past the 10th grade in high school math (I got side tracked for almost 30 years) would almost always think that the math 

$$A\cdot{B}$$

means $A$ times $B$. This is absolutely not true in this case.

The dot in the middle refers to the "dot product" as was mentioned in the start. So

$$A\cdot{B} \neq A\times{B}$$

instead... assuming that $A$ and $B$ are both 2D points or 2D vectors

$$A\cdot{B} = (A_x)(B_x)+(A_y)(B_y)$$

The magic of this is that when you multiply two vectors with each other, you actually get the slope of the line which is actually a single number, not a vector or point.

So, let's go a little further

$$0=\sum_{i=0}^{n}c_iB_i^n(t) \cdot{\sum_{i=0}^{n-1}d_jB_j^{n-1}(t)}$$

$$0=\sum_{i=0}^{n}\sum_{j=0}^{n-1}c_i\cdot{d_jB_i^n(t)B_j^{n-1}(t)}$$

It's becoming pretty clear now that we're nesting loops. We now have a loop for $i$ from $0$ to $n$ and another $j$ for $0$ to $n-1$.

Let's see a little code to represent it. This is just pseudo-code, but it should make it clearer

```python
n = 3
sum = 0
for i in range(0,n)
  for j in range(0,n-1)
    sum += dotproduct(c[i], d * B(i, n) * t * B(j, n - 1) * t)
```

Now we'll expand out the $B_i^n$ and $B_j^{n-1}$ stuff.

Remember

$$B_i^n(t) = {n \choose i}t^i(1-t)^{n-i}, i=0,...,n,$$

So, when substituting here we'll expand out to

$$0=\sum_{i=0}^{n}\sum_{j=0}^{n-1}c_i\cdot{d_j{n \choose i}(1-t)^{n-i}t^i{{n-1} \choose {j}}(1-t)^{n-1-j}t^j}$$

What we're doing here is like manually inlining functions in code. As you can tlel, the math gets messy, but it's really not that hard to follow.

Let's simplify a little bit. If we multiply the exponents out a little, we'll be able to see a little more clearly. If you're not sure about how this step works, check out over at [Khan Academy](https://www.khanacademy.org/math/algebra/x2f8bb11595b61c86:rational-exponents-radicals/x2f8bb11595b61c86:exponent-properties-review/v/multiplying-and-dividing-powers-with-integer-exponents)

We know that multiplying exponents is simply adding them together. So where we have 

$$(1-t)^{n-i}(1-t)^{n-1-j}$$

We can simply add

$$(n-i) + (n-1-j)$$

$$=n-i+n-1-j$$

$$=2n-1-i-j$$

$$=(2n-1)-(i+j)$$

Which gives us

$$(1-t)^{(2n-1)-(i+j)}$$

A little more simplification (and reordering) of the rest comes to

$$0=\sum_{i=0}^{n}\sum_{j=0}^{n-1}c_i\cdot{d_j{n \choose i}{{n-1} \choose {j}}t^{i+j}(1-t)^{(2n-1)-(i+j)}}$$

### One painful step for math kind

Let's take a look at the equation above with some parenthesis compared to the $B$ with the names of some of the variables changed to make it a little more visible... we'll add some color for good luck.

$$\color{gray}{0=\sum_{i=0}^{n}\sum_{j=0}^{n-1}c_i}\cdot{\color{gray}{d_j{n \choose i}{{n-1} \choose {j}}}[t^{\color{blue}{i+j}}(1-t)^{{\color{red}{(2n-1)}}-(\color{blue}{i+j})}]}$$

$$B_{\color{blue}{q}}^{\color{red}{p}}(t) = {{\color{red}{p}} \choose \color{blue}{q}}t^{\color{blue}{q}}(1-t)^{{\color{red}{p}}-{\color{blue}{q}}}$$

If we were to use $B_{\color{blue}{q}}^{\color{red}{p}}$ and substitute 

$$\color{red}{p} = \color{red}{2n-1}$$
$$\color{blue}{q} = \color{blue}{i+j}$$
    
We'd end up with
$$B_{\color{blue}{i+j}}^{\color{red}{2n-1}}(t) = {{\color{red}{2n-1}} \choose \color{blue}{i+j}}t^{\color{blue}{i+j}}(1-t)^{{\color{red}{(2n-1)}}-{\color{blue}{(i+j)}}}$$

Which is really close to

$$t^{\color{blue}{i+j}}(1-t)^{{\color{red}{(2n-1)}}-(\color{blue}{i+j})}$$

Which is in the larger equation above. In fact, if we were to device it by the choose, you'd see

$$\frac{{{\color{red}{2n-1}} \choose \color{blue}{i+j}}t^{\color{blue}{i+j}}(1-t)^{{\color{red}{(2n-1)}}-{\color{blue}{(i+j)}}}}{{{\color{red}{2n-1}} \choose \color{blue}{i+j}}} = t^{\color{blue}{i+j}}(1-t)^{{\color{red}{(2n-1)}}-(\color{blue}{i+j})}$$

So we could rewrite the larger equation as

$$\color{gray}{0=\sum_{i=0}^{n}\sum_{j=0}^{n-1}c_i}\cdot{\color{gray}{d_j{n \choose i}{{n-1} \choose {j}}}\frac{{{\color{red}{2n-1}} \choose \color{blue}{i+j}}t^{\color{blue}{i+j}}(1-t)^{{\color{red}{(2n-1)}}-{\color{blue}{(i+j)}}}}{{{\color{red}{2n-1}} \choose \color{blue}{i+j}}}}$$

And move the division a little to the left

$$\color{gray}{0=\sum_{i=0}^{n}\sum_{j=0}^{n-1}c_i}\cdot{\color{gray}{d_j}\frac{{n \choose i}{{n-1} \choose {j}}}{{\color{red}{2n-1}} \choose \color{blue}{i+j}}{{\color{red}{2n-1}} \choose \color{blue}{i+j}} t^{\color{blue}{i+j}}(1-t)^{{\color{red}{(2n-1)}}-(\color{blue}{i+j})}}$$

Which can be rewritten in terms of $B_{i}^{n}$ as

$$0=\sum_{i=0}^{n}\sum_{j=0}^{n-1}c_i\cdot{d_j\frac{{n \choose i}{{n-1} \choose {j}}}{{\color{red}{2n-1}} \choose \color{blue}{i+j}}}B_{\color{blue}{i+j}}^{\color{red}{2n-1}}(t)$$

### Take a step back to appreciate the beauty of where we've gotten to so far

A beautiful thing about where we've come to is that the entire chunk of math seen here

$$\frac{{n \choose i}{{n-1} \choose {j}}}{{\color{red}{2n-1}} \choose \color{blue}{i+j}}$$

Can be precalculated for all values of $i$ and $j$. In fact it will fit BEAUTIFULLY into a 2D array of constants.

In [98]:
import numpy as np
from scipy.special import comb
import pandas as pd

n = 3

coeffs = np.empty([n + 1, n])

for i in range(0, n+1):
    for j in range(0, n):
        coeffs[i,j] = scipy.special.comb(n, i) * scipy.special.comb(n - 1, j) / scipy.special.comb(2 * n - 1, i + j)
    
pd.DataFrame(data=coeffs)

,0,1,2
0,1.0,0.4,0.1
1,0.6,0.6,0.3
2,0.3,0.6,0.6
3,0.1,0.4,1.0


Philip J. Schneider recommends in his math that the table we just generated is renamed to the variable named $z_{i.j}$

I will mention that if you're reading his code, he reversed the axis of the table. I decided to keep $i$ and $j$ where they seem to belong in the math.

$$0=\sum_{i=0}^{n}\sum_{j=0}^{n-1}c_i\cdot{d_jz_{i,j}}B_{\color{blue}{i+j}}^{\color{red}{2n-1}}(t)$$

TODO: Ask Munib or Fabian to explain why this next step is ok

And then we can group things together a little further as a refactoring which will look pretty nice in code. Let's make a new variable as follows:

$$\color{green}{w_{i,j}}=c_i\cdot{d_jz_{i,j}}$$

$$0=\sum_{i=0}^{n}\sum_{j=0}^{n-1}\color{green}{w_{i,j}}B_{\color{blue}{i+j}}^{\color{red}{2n-1}}(t)$$

## Chapter summary (not even close to done)

This is pretty much the end of that chapter in the book, but it leaves us with a lot of stuff.

Let me summarize all the stuff we have done so far into once place. In fact, you could have just skipped to here to get all the math without any appreciation for how we got here.

$$0=\sum_{i=0}^{n}\sum_{j=0}^{n-1}\color{green}{w_{i,j}}B_{\color{blue}{i+j}}^{\color{red}{2n-1}}(t)$$

$$\color{green}{w_{i,j}}=\color{orange}{c_i}\cdot{\color{purple}{d_j}\color{brown}{z_{i,j}}}$$

$$\color{brown}{z_{i,j}} = \frac{{n \choose i}{{n-1} \choose {j}}}{{2n-1} \choose {i+j}}$$

$$B_{\color{blue}{q}}^{\color{red}{p}}(t) = {{\color{red}{p}} \choose \color{blue}{q}}t^{\color{blue}{q}}(1-t)^{{\color{red}{p}}-{\color{blue}{q}}}$$

$$\color{purple}{d_j}=n(V_{j+1}-V_j)$$

$$\color{orange}{c_i}=V_i-P$$

So, that's pretty much the whole thing.

I think in order to make code out of this, we need to expand out as much as possible since even though SciPy actually has a solver, it would be horribly inefficient to depend on it for this.

Let's do some code now to get some of the pieces together.

![Code Example Curve](CodeExampleCurve.png)

Assuming the curve seen above, let's initialize some data. I've intentionally been more verbose than necessary.

In [99]:
x0 = 1.0
y0 = 1.6
x1 = 2.8
y1 = 1
x2 = 1
y2 = 1
x3 = 2.6
y3 = 1.4

V = np.array([
    [x0, y0],
    [x1, y1],
    [x2, y2],
    [x3, y3]
])

xp = -0.6
yp = 1.8

P = np.array([xp, yp])

c = V - P

pd.DataFrame(data=c)

,0,1
0,1.6,-0.2
1,3.4,-0.8
2,1.6,-0.8
3,3.2,-0.4


So, we have created the array of points called $c$ and have more or less eliminated $P$ from the math. Leaving us with

$$0=\sum_{i=0}^{n}\sum_{j=0}^{n-1}\color{green}{w_{i,j}}B_{\color{blue}{i+j}}^{\color{red}{2n-1}}(t)$$

$$\color{green}{w_{i,j}}=\color{orange}{c_i}\cdot{\color{purple}{d_j}\color{brown}{z_{i,j}}}$$

$$\color{brown}{z_{i,j}} = \frac{{n \choose i}{{n-1} \choose {j}}}{{2n-1} \choose {i+j}}$$

$$B_{\color{blue}{q}}^{\color{red}{p}}(t) = {{\color{red}{p}} \choose \color{blue}{q}}t^{\color{blue}{q}}(1-t)^{{\color{red}{p}}-{\color{blue}{q}}}$$

$$\color{purple}{d_j}=n(V_{j+1}-V_j)$$

From this point forward, since we know that $n=3$ we should substitute it everyplace we need it. So, let's create a variable called $z$ which is really $\color{brown}{z_{i,j}}$. We had done it above, but I want to hardcode the value $n=3$.

In [100]:
z = np.empty([4, 3])

for i in range(0, 4):
    for j in range(0, 3):
        z[i,j] = scipy.special.comb(3, i) * scipy.special.comb(2, j) / scipy.special.comb(6 - 1, i + j)
    
pd.DataFrame(data=z)

,0,1,2
0,1.0,0.4,0.1
1,0.6,0.6,0.3
2,0.3,0.6,0.6
3,0.1,0.4,1.0


In fact, in production code, we would simply initialize the array using constants.

So, now we have

$$0=\sum_{i=0}^{3}\sum_{j=0}^{2}\color{green}{w_{i,j}}B_{\color{blue}{i+j}}^{\color{red}{2(3)-1}}(t)=\sum_{i=0}^{3}\sum_{j=0}^{2}\color{green}{w_{i,j}}B_{\color{blue}{i+j}}^{\color{red}{5}}(t)$$

$$\color{green}{w_{i,j}}=\color{orange}{c_i}\cdot{\color{purple}{d_j}\color{brown}{z_{i,j}}}$$

$$B_{\color{blue}{q}}^{\color{red}{p}}(t) = {{\color{red}{p}} \choose \color{blue}{q}}t^{\color{blue}{q}}(1-t)^{{\color{red}{p}}-{\color{blue}{q}}}$$

$$\color{purple}{d_j}=3(V_{j+1}-V_j)$$

Another low hanging fruit in the code is to solve for $\color{purple}{d_j}$ which will produce an array for values $0$, $1$, and $2$ and be used repetitively.

In [101]:
d = np.empty([3,2])

d[0] = 3.0 * (V[1] - V[0])
d[1] = 3.0 * (V[2] - V[1])
d[2] = 3.0 * (V[3] - V[2])

pd.DataFrame(data=d)

,0,1
0,5.4,-1.8
1,-5.4,0.0
2,4.8,1.2


What we've done with $d$ is to solve $3$ times the deltas of each point.

Consider $c_1$ and $c_0$.

$$C_0 = \begin{bmatrix}1.6\\ -0.2\end{bmatrix}$$

$$C_1 = \begin{bmatrix}3.4\\-0.8\end{bmatrix}$$

$$C_1-C_0 = \begin{bmatrix}3.4\\-0.8\end{bmatrix} - \begin{bmatrix}1.6\\ -0.2\end{bmatrix} = \begin{bmatrix}1.8\\ -0.6\end{bmatrix}$$

Now multiply by 3

$$3C_1-C_0 = 3\begin{bmatrix}1.8\\ -0.6\end{bmatrix}= \begin{bmatrix}5.4\\ -1.8\end{bmatrix}$$

And you see the code generated above shows that $d_0$ is precisely that

Moving on, we've produced code for quite a bit already. So, that leaves us with

$$0=\sum_{i=0}^{3}\sum_{j=0}^{2}\color{green}{w_{i,j}}B_{\color{blue}{i+j}}^{\color{red}{5}}(t)$$

$$\color{green}{w_{i,j}}=\color{orange}{c_i}\cdot{\color{purple}{d_j}\color{brown}{z_{i,j}}}$$

$$B_{\color{blue}{q}}^{\color{red}{p}}(t) = {{\color{red}{p}} \choose \color{blue}{q}}t^{\color{blue}{q}}(1-t)^{{\color{red}{p}}-{\color{blue}{q}}}$$

The next piece of code will solve for all values of $\color{green}{w_{i,j}}$

In [102]:
w = np.empty([4,3])

for i in range(0, 4):
    for j in range(0, 3):
        w[i,j] = np.dot(c[i], d[j])
        
pd.DataFrame(data=w)

,0,1,2
0,9.00,-8.64,7.44
1,19.80,-18.36,15.36
2,10.08,-8.64,6.72
3,18.00,-17.28,14.88


In [103]:
w = np.empty([4,3])

for i in range(0, 4):
    for j in range(0, 3):
        w[i,j] = np.dot(c[i], d[j])*z[i,j]
        
pd.DataFrame(data=w)

,0,1,2
0,9.000,-3.456,0.744
1,11.880,-11.016,4.608
2,3.024,-5.184,4.032
3,1.800,-6.912,14.880


I love giving a description of what every step is but it's beginning to make more sense in the math rather than the code. Remember though tht the dot product is a relationship between the angle between two vectors as was seen near the top of this document.

Winding down, we're left with

$$0=\sum_{i=0}^{3}\sum_{j=0}^{2}\color{green}{w_{i,j}}B_{\color{blue}{i+j}}^{\color{red}{5}}(t)$$

$$B_{\color{blue}{q}}^{\color{red}{p}}(t) = {{\color{red}{p}} \choose \color{blue}{q}}t^{\color{blue}{q}}(1-t)^{{\color{red}{p}}-{\color{blue}{q}}}$$

So let's start combining the math here

$$0=\sum_{i=0}^{3}\sum_{j=0}^{2}\color{green}{w_{i,j}}{{\color{red}{5}} \choose \color{blue}{i+j}}t^{\color{blue}{i+j}}(1-t)^{{\color{red}{5}}-{\color{blue}{i+j}}}$$

### Where did all the vectors go?

Notice that there are NO vectors left. We have completely eliminated any of the super fancy linear algebra or maybe you might called it "matrix math" from the equation. $\color{green}{w_{i,j}}$ is just an array of numbers. In reality, all we have left for variables is $t$ once we've plugged in all the other stuff.

### Seriously reducing our possible options

Another thing which is interesting that $\color{blue}{i+j}$ shows up a whole lot, but there aren't so many values for that. After all, there are only so many possibly numbers which $i$ and $j$ can add up to. In fact, if you think about it, all the values of $i$ and all the values of $j$ can never equal more than $5$ when they're added up. So, we know that $\color{blue}{i+j}$ will always be $0$, $1$, $2$, $3$, $4$, or $5$. That's only 6 possible values!

So, we can actually calculate all the possible formulas of what used to be $B$

Let's first get a list of the $n \choose k$ stuff I'm going to make a new variable called $r$ which will represent

$$\color{magenta}{r_{i+j}}= {{\color{red}{5}} \choose \color{blue}{i+j}}$$

So that will change our formula to

$$0=\sum_{i=0}^{3}\sum_{j=0}^{2}\color{green}{w_{i,j}}\color{magenta}{r_{i+j}}t^{\color{blue}{i+j}}(1-t)^{{\color{red}{5}}-{\color{blue}{i+j}}}$$


In [104]:
r = np.empty([6])

for k in range(0, 6):
    r[k] = scipy.special.comb(5, k)
    
pd.DataFrame(data=r)

,0
0,1.0
1,5.0
2,10.0
3,10.0
4,5.0
5,1.0


I suppose I didn't need to make a new variable for this, but I did anyway. But we can now make a new array called $\color{purple}{s}$ which precalculates all possible values of $\color{green}{w_{i,j}}\color{magenta}{r_{i+j}}$

TODO: DANGER WILL ROBINSON!!! The code provided with the Graphics Gems book completely ignores $r_{i+j}$, it would be a good idea to try the algorithm both with and without it. To align with the book, I've commented our the $r_{i+j}$ step.

In [105]:
s = np.empty([4,3])

for i in range(0,4):
    for j in range(0,3):
        s[i,j] = w[i,j] #*r[i+j]

pd.DataFrame(data=s)

,0,1,2
0,9.000,-3.456,0.744
1,11.880,-11.016,4.608
2,3.024,-5.184,4.032
3,1.800,-6.912,14.880


## Ok... so we've trimmed it down to almost nothing! Let's see what we have left

$$0=\sum_{i=0}^{3}\sum_{j=0}^{2}\color{purple}{s_{i,j}}(t^{\color{blue}{i+j}}(1-t)^{{\color{red}{5}}-{\color{blue}{i+j}}})$$

This is where things get a little more interesting. What if we were to write out the entire equation now and get rid of that ugly looking $\sum_{i=0}^{3}\sum_{j=0}^{2}$. I know... we don't really want to do that.. it'll be gigantic, but I'll do it to make a point.

I think it should be obvious that there are only 6 possible ways to write $t^{\color{blue}{i+j}}(1-t)^{{\color{red}{5}}-{\color{blue}{i+j}}}$ in the full equation.

$$t^{0}(1-t)^{{5}-{0}} = t^{0}(1-t)^{5} = \color{magenta}{(1-t)^{5}}$$

$$t^{1}(1-t)^{{5}-{1}} = \color{red}{t^1(1-t)^4}$$

$$t^{2}(1-t)^{{5}-{2}} = \color{blue}{t^2(1-t)^3}$$

$$t^{3}(1-t)^{{5}-{3}} = \color{green}{t^3(1-t)^2}$$

$$t^{4}(1-t)^{{5}-{4}} = \color{purple}{t^4(1-t)^1}$$

$$t^{5}(1-t)^{{5}-{5}} = t^{5}(1-t)^{0} = \color{brown}{t^{5}}$$

If we draw the equation out as a table and we list all the things we need to add together, we should see a pattern

$0 = $

| - | --------- j=0 --------- | --------- j=1 --------- | --------- j=2 --------- |
|-|--------------------|---------------------------------------|---------------------------------------|
| i=0 |  $s_{0,0}\color{magenta}{t^0(1-t)^5}$ | $+s_{0,1}\color{red}{t^1(1-t)^4}$ | $+s_{0,2}\color{blue}{t^2(1-t)^3}$ |
| i=1 | $+s_{1,0}\color{red}{t^1(1-t)^4}$ | $+s_{1,1}\color{blue}{t^2(1-t)^3}$ | $+s_{1,2}\color{green}{t^3(1-t)^2}$ |
| i=2 | $+s_{2,0}\color{blue}{t^2(1-t)^3}$ | $+s_{2,1}\color{green}{t^3(1-t)^2}$ | $+s_{2,2}\color{purple}{t^4(1-t)^1}$ |
| i=3 | $+s_{3,0}\color{green}{t^3(1-t)^2}$ | $+s_{3,1}\color{purple}{t^4(1-t)^1}$ | $+s_{3,2}\color{brown}{t^5(1-t)^0}$ |

So, if we write it out algebraically while grouping common factors

$$0 =\\
s_{0,0}\color{magenta}{t^0(1-t)^5} +\\
s_{0,1}\color{red}{t^1(1-t)^4} + s_{1,0}\color{red}{t^1(1-t)^4} +\\
s_{0,2}\color{blue}{t^2(1-t)^3} + s_{1,1}\color{blue}{t^2(1-t)^3} + s_{2,0}\color{blue}{t^2(1-t)^3} +\\
s_{1,2}\color{green}{t^3(1-t)^2} + s_{2,1}\color{green}{t^3(1-t)^2} + s_{3,0}\color{green}{t^3(1-t)^2} +\\
s_{2,2}\color{purple}{t^4(1-t)^1} + s_{3,1}\color{purple}{t^4(1-t)^1}\\
s_{3,2}\color{brown}{t^5(1-t)^0}$$

Now factor a little

$$0=\\
s_{0,0}\color{magenta}{t^0(1-t)^5} +\\
(s_{0,1}+s_{1,0})\color{red}{t^1(1-t)^4} + \\
(s_{0,2} + s_{1,1} + s_{2,0})\color{blue}{t^2(1-t)^3} +\\
(s_{1,2} + s_{2,1} + s_{3,0})\color{green}{t^3(1-t)^2} +\\
(s_{2,2} + s_{3,1})\color{purple}{t^4(1-t)^1}\\
s_{3,2}\color{brown}{t^5(1-t)^0}
$$

Let's make a new symbol $W_k$ which will represent the coefficients needed for the following equation

$$
W_0\color{magenta}{t^0(1-t)^5} +
W_1\color{red}{t^1(1-t)^4} +
W_2\color{blue}{t^2(1-t)^3} +
W_3\color{green}{t^3(1-t)^2} +
W_4\color{purple}{t^4(1-t)^1} +
W_5\color{brown}{t^5(1-t)^0}
$$

And here's the code to do precisely the same thing

In [107]:
W = np.array([
    s[0,0],
    s[0,1] + s[1,0],
    s[0,2] + s[1,1] + s[2,0],
    s[1,2] + s[2,1] + s[3,0],
    s[2,2] + s[3,1],
    s[3,2]
])

pd.DataFrame(data=W)

,0
0,9.000
1,8.424
2,-7.248
3,1.224
4,-2.880
5,14.880


So if we were to write out the equation long hand, we would see what is generated by the code below

In [114]:
from IPython.display import Markdown as md
str = "$$f(t)=({:.{prec}})t^0 (1-t)^5 + ({:.{prec}})t^1 (1-t)^4 + ({:.{prec}})t^2 (1-t)^3 + ({:.{prec}})t^3 (1-t)^2 + ({:.{prec}})t^4 (1-t)^1 +({:.{prec}})t^5 (1-t)^0$$".format(W[0],W[1],W[2],W[3],W[4],W[5],prec=3)
md(str)

$$f(t)=(9.0)t^0 (1-t)^5 + (8.42)t^1 (1-t)^4 + (-7.25)t^2 (1-t)^3 + (1.22)t^3 (1-t)^2 + (-2.88)t^4 (1-t)^1 +(14.9)t^5 (1-t)^0$$

I used Python to produce the document here because it allows me to automatically update the document.



So we now have a 5th order polynomial as prescribed by the chapter denoted as "Solving the nearest-point-on-curve problem".

![Failure is an option](PlotOfCodeExample_NoRoots.png)

It may look from this image that we nailed it and the solution to the equation is an $x$ axis intersect at point $1$. Well... no, it wasn't. This problem actually has no solutions. But it will allow me to refactor my code into a function with a new example that does have a solution.

